In [287]:
import string
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.naive_bayes import MultinomialNB
import numpy as np
from sklearn.metrics import classification_report
import os
import urllib.request   
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM,Flatten
import numpy as np
import gensim
from gensim import corpora, models, similarities
from gensim.parsing.preprocessing import remove_stopwords    
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from sklearn import tree
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.ensemble import RandomForestClassifier

In [2]:
if not os.path.exists('data'):
    os.mkdir('data')

urllib.request.urlretrieve('https://www.gutenberg.org/files/2591/2591-0.txt', 'data/Jacob and Wilhelm/grimms_fairy_tales.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/345/pg345.txt', 'data/Bram Stoker/dracula.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/84/pg84.txt', 'data/Mary Shelley/frankenstein.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/files/2701/2701-0.txt', 'data/Herman Melville/moby_dick.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/files/74/74-0.txt', 'data/Mark Twain/tom_sawyer.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/38189/pg38189.txt', 'data/Ambrose Blacklock/A_TREATISE_ON_SHEEP.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/2166/pg2166.txt', 'data/H.Rider Haggard/King_Solomons_Mines.txt')
urllib.request.urlretrieve('http://www.gutenberg.org/cache/epub/21728/pg21728.txt', 'data/R.M.Ballantyne/THE_DOG_CRUSOE_AND_HIS_MASTER.txt')
urllib.request.urlretrieve('https://www.gutenberg.org/files/95/95-0.txt', 'data/Anthony Hope/THE_PRISONER_OF_ZENDA.txt')
urllib.request.urlretrieve('https://www.gutenberg.org/files/1257/1257-0.txt', 'data/Alexandre Dumas/THE_THREE_MUSKETEERS.txt')


('data/Alexandre Dumas/THE_THREE_MUSKETEERS.txt',
 <http.client.HTTPMessage at 0x18a7dc06a20>)

In [288]:
path = './data'
token_dict = {}
dfs=[]



for dirpath, dirs, files in os.walk(path):
    for f in files:
        fname = os.path.join(dirpath, f)
        fdir=os.path.basename(dirpath)
        print ("fname=", fname)
        dt=pd.read_fwf(fname, sep=".", header=None)
        dt['label']=fdir
        dfs.append(dt)

            
big_frame = pd.concat(dfs, ignore_index=True)      

fname= ./data\Alexandre Dumas\THE_THREE_MUSKETEERS.txt
fname= ./data\Ambrose Blacklock\A_TREATISE_ON_SHEEP.txt
fname= ./data\Anthony Hope\THE_PRISONER_OF_ZENDA.txt
fname= ./data\Bram Stoker\dracula.txt
fname= ./data\H.Rider Haggard\King_Solomons_Mines.txt
fname= ./data\Herman Melville\moby_dick.txt
fname= ./data\Jacob and Wilhelm\grimms_fairy_tales.txt
fname= ./data\Mark Twain\tom_sawyer.txt
fname= ./data\Mary Shelley\frankenstein.txt
fname= ./data\R.M.Ballantyne\THE_DOG_CRUSOE_AND_HIS_MASTER.txt


C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [289]:
big_frame.head()

,0,label,1
0,ï»¿ THE THREE MUSKETEERS,Alexandre Dumas,NaN
1,This ebook is for the use of anyone anywhere i...,Alexandre Dumas,NaN
2,most other parts of the world at no cost and w...,Alexandre Dumas,NaN
3,"whatsoever. You may copy it, give it away or r...",Alexandre Dumas,NaN
4,of the Project Gutenberg License included with...,Alexandre Dumas,NaN


In [290]:
big_frame["label"].value_counts()

Alexandre Dumas      22602
Herman Melville      19222
Bram Stoker          13414
Jacob and Wilhelm     9571
R.M.Ballantyne        7830
H.Rider Haggard       7451
Mark Twain            6927
Ambrose Blacklock     6577
Anthony Hope          5206
Mary Shelley          1300
Name: label, dtype: int64

In [291]:
big_frame.dtypes
big_frame[0] = big_frame[0].replace('"', ' ', regex=True)
big_frame[0] = big_frame[0].replace('!', ' ', regex=True)
big_frame[0] = big_frame[0].replace('-', ' ', regex=True)
big_frame[0] = big_frame[0].replace(',', ' ', regex=True)
big_frame[0] = big_frame[0].replace('_', ' ', regex=True)
big_frame[0] = big_frame[0].replace(';', ' ', regex=True)
big_frame[0] = big_frame[0].replace('»', ' ', regex=True)
big_frame[0] = big_frame[0].replace('@', ' ', regex=True)
big_frame[0] = big_frame[0].replace('$', ' ', regex=True)
big_frame[0] = big_frame[0].replace('&', ' ', regex=True)
big_frame[0] = big_frame[0].replace('%', ' ', regex=True)
big_frame[0] = big_frame[0].replace('â€™', ' ', regex=True)
big_frame[0] = big_frame[0].replace('â€�', ' ', regex=True)



In [292]:
df_x=big_frame[0].values.astype('U')
df_y=big_frame["label"]


## 1. Word tokenize+TF-IDF

In [293]:

stop_words = stopwords.words('english') + list(punctuation)
 
def tokenize(text):
    words = word_tokenize(text)
    words = [w.lower() for w in words]
    return [w for w in words if w not in stop_words and not w.isdigit()]



## 1.1 Naive Bayes classifier 

In [294]:

cv = TfidfVectorizer(tokenizer=tokenize,min_df=1,ngram_range =(1,2))
x_trainc=cv.fit_transform(df_x)
x_train, x_test, y_train, y_test = train_test_split(x_trainc, df_y, test_size=0.1, random_state=4)
mnb = MultinomialNB(alpha=0.1)
clf=mnb.fit(x_train,y_train)


In [295]:
predicted = clf.predict(x_test)

In [296]:
print(classification_report(y_test, predicted))

                   precision    recall  f1-score   support

  Alexandre Dumas       0.66      0.88      0.75      2266
Ambrose Blacklock       0.74      0.66      0.70       638
     Anthony Hope       0.86      0.30      0.45       515
      Bram Stoker       0.60      0.69      0.64      1348
  H.Rider Haggard       0.74      0.41      0.53       778
  Herman Melville       0.70      0.81      0.75      1907
Jacob and Wilhelm       0.78      0.75      0.77       950
       Mark Twain       0.85      0.50      0.63       665
     Mary Shelley       1.00      0.84      0.92       135
   R.M.Ballantyne       0.76      0.57      0.65       808

      avg / total       0.72      0.70      0.69     10010



## 1.2 Stochastic Gradient Descent classifier

In [297]:
svm=SGDClassifier()
svm_classifier=svm.fit(x_train,y_train)

C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [298]:
svm_predicted = svm_classifier.predict(x_test)

In [299]:
print(classification_report(y_test, svm_predicted))

                   precision    recall  f1-score   support

  Alexandre Dumas       0.68      0.85      0.75      2266
Ambrose Blacklock       0.68      0.70      0.69       638
     Anthony Hope       0.77      0.36      0.49       515
      Bram Stoker       0.64      0.61      0.63      1348
  H.Rider Haggard       0.70      0.40      0.51       778
  Herman Melville       0.65      0.79      0.71      1907
Jacob and Wilhelm       0.70      0.75      0.73       950
       Mark Twain       0.68      0.48      0.57       665
     Mary Shelley       0.97      0.79      0.87       135
   R.M.Ballantyne       0.71      0.53      0.60       808

      avg / total       0.68      0.68      0.67     10010



## 1.3 Decision Tree and Random Forest

In [300]:
x_train, x_test, y_train, y_test = train_test_split(x_trainc, df_y, test_size=0.2, random_state=4)
Tree_clf = tree.DecisionTreeClassifier()
Tree_clf = Tree_clf.fit(x_train, y_train)


In [301]:
predicted = Tree_clf.predict(x_test)
print(classification_report(y_test, predicted))

                   precision    recall  f1-score   support

  Alexandre Dumas       0.60      0.65      0.62      4513
Ambrose Blacklock       0.44      0.49      0.46      1294
     Anthony Hope       0.43      0.30      0.36      1023
      Bram Stoker       0.33      0.43      0.37      2677
  H.Rider Haggard       0.38      0.29      0.33      1489
  Herman Melville       0.57      0.60      0.59      3846
Jacob and Wilhelm       0.55      0.50      0.52      1935
       Mark Twain       0.47      0.36      0.41      1418
     Mary Shelley       1.00      0.78      0.87       272
   R.M.Ballantyne       0.50      0.40      0.44      1553

      avg / total       0.51      0.50      0.50     20020



In [302]:
RF_clf = RandomForestClassifier(
n_estimators=10,max_depth=40, random_state=0)

In [303]:
RF_clf=RF_clf.fit(x_train,y_train)
predicted = Tree_clf.predict(x_test)
print(classification_report(y_test, predicted))

                   precision    recall  f1-score   support

  Alexandre Dumas       0.60      0.65      0.62      4513
Ambrose Blacklock       0.44      0.49      0.46      1294
     Anthony Hope       0.43      0.30      0.36      1023
      Bram Stoker       0.33      0.43      0.37      2677
  H.Rider Haggard       0.38      0.29      0.33      1489
  Herman Melville       0.57      0.60      0.59      3846
Jacob and Wilhelm       0.55      0.50      0.52      1935
       Mark Twain       0.47      0.36      0.41      1418
     Mary Shelley       1.00      0.78      0.87       272
   R.M.Ballantyne       0.50      0.40      0.44      1553

      avg / total       0.51      0.50      0.50     20020



In [304]:
big_frame = pd.get_dummies(big_frame, columns=['label'])

In [305]:
big_frame

,0,1,label_Alexandre Dumas,label_Ambrose Blacklock,label_Anthony Hope,label_Bram Stoker,label_H.Rider Haggard,label_Herman Melville,label_Jacob and Wilhelm,label_Mark Twain,label_Mary Shelley,label_R.M.Ballantyne
0,ï ¿ THE THREE MUSKETE...,NaN,1,0,0,0,0,0,0,0,0,0
1,This ebook is for the use of anyone anywhere i...,NaN,1,0,0,0,0,0,0,0,0,0
2,most other parts of the world at no cost and w...,NaN,1,0,0,0,0,0,0,0,0,0
3,whatsoever. You may copy it give it away or r...,NaN,1,0,0,0,0,0,0,0,0,0
4,of the Project Gutenberg License included with...,NaN,1,0,0,0,0,0,0,0,0,0
5,http://www.gutenberg.org/license. If you are n...,NaN,1,0,0,0,0,0,0,0,0,0
6,States you ll have to check the laws of the c...,NaN,1,0,0,0,0,0,0,0,0,0
7,located before using this ebook.,NaN,1,0,0,0,0,0,0,0,0,0
8,Title: The Three Musketeers,NaN,1,0,0,0,0,0,0,0,0,0
9,Author: Alexandre Dumas Pere,NaN,1,0,0,0,0,0,0,0,0,0


In [306]:
big_frame.columns

Index([                        0,                         1,
         'label_Alexandre Dumas', 'label_Ambrose Blacklock',
            'label_Anthony Hope',       'label_Bram Stoker',
         'label_H.Rider Haggard',   'label_Herman Melville',
       'label_Jacob and Wilhelm',        'label_Mark Twain',
            'label_Mary Shelley',    'label_R.M.Ballantyne'],
      dtype='object')

In [307]:
feature_cols=['label_Alexandre Dumas', 'label_Ambrose Blacklock',
            'label_Anthony Hope',       'label_Bram Stoker',
         'label_H.Rider Haggard',   'label_Herman Melville',
       'label_Jacob and Wilhelm',        'label_Mark Twain',
            'label_Mary Shelley',    'label_R.M.Ballantyne']
y=big_frame[feature_cols]
big_frame.to_csv('big_frame.csv', encoding='utf-8')
x_trainc=big_frame[0].values.astype('U')
max_words=40000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(x_trainc)
sequences = tokenizer.texts_to_sequences(x_trainc)

In [308]:
x_train, x_test, y_train, y_test = train_test_split(sequences, y, test_size=0.1, random_state=4)

## 1.4 RNN classifier

In [175]:
top_words = 45000   
sequence_length = 500
max_review_length=500

x_train = sequence.pad_sequences(x_train, maxlen=sequence_length, padding="post", truncating="post")
x_test = sequence.pad_sequences(x_test, maxlen=sequence_length, padding="post", truncating="post")
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(32))
model.add(Dense(10, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=3, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 500, 32)           1440000   
_________________________________________________________________
lstm_13 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_13 (Dense)             (None, 10)                330       
Total params: 1,448,650
Trainable params: 1,448,650
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/3
90090/90090 [==============================] - ETA: 0s - loss: 0.3080 - acc: 0.899 - 570s 6ms/step - loss: 0.3080 - acc: 0.8999
Epoch 2/3
90090/90090 [==============================] - 568s 6ms/step - loss: 0.3042 - acc: 0.9000
Epoch 3/3
90090/90090 [==============================] - 571s 6ms/step - loss: 0.3042 - acc: 0.9000
Accuracy: 90.00%


## 2. Countvectorizer

In [158]:
df_x=big_frame[0].values.astype('U')


In [281]:
count_vec = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 2), max_df=1.0, min_df=1, max_features=10000)

x = count_vec.fit_transform(df_x)


## 2.1 Naive Bayes

In [164]:
x_train, x_test, y_train, y_test = train_test_split(x, df_y, test_size=0.2, random_state=4)
mnb = MultinomialNB(alpha=0.1)
clf=mnb.fit(x_train,y_train)

In [165]:
predicted = clf.predict(x_test)

In [166]:
print(classification_report(y_test, predicted))

                   precision    recall  f1-score   support

  Alexandre Dumas       0.70      0.77      0.73      4513
Ambrose Blacklock       0.57      0.70      0.63      1294
     Anthony Hope       0.52      0.44      0.48      1023
      Bram Stoker       0.59      0.63      0.61      2677
  H.Rider Haggard       0.58      0.49      0.53      1489
  Herman Melville       0.70      0.72      0.71      3846
Jacob and Wilhelm       0.71      0.71      0.71      1935
       Mark Twain       0.65      0.49      0.56      1418
     Mary Shelley       0.29      0.10      0.15       272
   R.M.Ballantyne       0.64      0.59      0.61      1553

      avg / total       0.65      0.65      0.65     20020



## 2.2 SVM classifier

In [167]:
svm_classifier=svm.fit(x_train,y_train)
svm_predicted = svm_classifier.predict(x_test)
print(classification_report(y_test, svm_predicted))

C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


                   precision    recall  f1-score   support

  Alexandre Dumas       0.70      0.78      0.74      4513
Ambrose Blacklock       0.60      0.67      0.63      1294
     Anthony Hope       0.58      0.38      0.46      1023
      Bram Stoker       0.59      0.57      0.58      2677
  H.Rider Haggard       0.59      0.40      0.47      1489
  Herman Melville       0.66      0.73      0.69      3846
Jacob and Wilhelm       0.65      0.71      0.68      1935
       Mark Twain       0.65      0.43      0.52      1418
     Mary Shelley       0.38      0.92      0.54       272
   R.M.Ballantyne       0.66      0.53      0.59      1553

      avg / total       0.64      0.64      0.63     20020



## 2.3 Decision Tree

In [168]:
Tree_clf = tree.DecisionTreeClassifier()
Tree_clf = Tree_clf.fit(x_train, y_train)
predicted = Tree_clf.predict(x_test)
print(classification_report(y_test, predicted))

                   precision    recall  f1-score   support

  Alexandre Dumas       0.53      0.69      0.60      4513
Ambrose Blacklock       0.44      0.51      0.47      1294
     Anthony Hope       0.35      0.30      0.33      1023
      Bram Stoker       0.42      0.39      0.40      2677
  H.Rider Haggard       0.38      0.32      0.35      1489
  Herman Melville       0.59      0.51      0.55      3846
Jacob and Wilhelm       0.54      0.47      0.50      1935
       Mark Twain       0.47      0.33      0.39      1418
     Mary Shelley       0.29      0.97      0.44       272
   R.M.Ballantyne       0.53      0.39      0.45      1553

      avg / total       0.49      0.49      0.48     20020



In [172]:
df_x=big_frame[0].values.astype('U')

count_vec = CountVectorizer(stop_words="english", analyzer='word', 
                            ngram_range=(1, 2), max_df=1.0, min_df=1, max_features=5000)

x = count_vec.fit_transform(df_x)

top_words = 10000   
sequence_length = 500
max_review_length=500

x_train, x_test, y_train, y_test = train_test_split(x.toarray(), y, test_size=0.2, random_state=4)

x_train = sequence.pad_sequences(x_train, maxlen=sequence_length, padding="post", truncating="post")
x_test = sequence.pad_sequences(x_test, maxlen=sequence_length, padding="post", truncating="post")
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(LSTM(32))
model.add(Dense(10, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(x_train, y_train, epochs=4, batch_size=32)
# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (None, 500, 32)           320000    
_________________________________________________________________
lstm_12 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dense_12 (Dense)             (None, 10)                330       
Total params: 328,650
Trainable params: 328,650
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/4
80080/80080 [==============================] - 481s 6ms/step - loss: 0.3090 - acc: 0.8985
Epoch 2/4
80080/80080 [==============================] - 473s 6ms/step - loss: 0.3043 - acc: 0.9000
Epoch 3/4
80080/80080 [==============================] - 472s 6ms/step - loss: 0.3043 - acc: 0.9000
Epoch 4/4
80080/80080 [==============================] - 469s 6ms/step - loss: 0.3042 - acc: 0.

## 3. Word2vec

In [315]:
big_frame['tokenized_column'] = big_frame[0].fillna("").map(word_tokenize)
stop = stopwords.words('english')
x=big_frame['tokenized_column'].apply(lambda x: [item for item in x if item not in stop])


In [316]:
model = gensim.models.Word2Vec(x.tolist(), size=500)
w2v = dict(zip(model.wv.index2word, model.wv.syn0))

C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  


In [317]:
X_new = np.array([np.mean([w2v[w] for w in words if w in w2v], axis=0)for words in x])

C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\User\Anaconda3\envs\tensorflow1\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [319]:
big_frame['w2v']=X_new
X=big_frame['w2v']
X.head()

0    [0.08253203, -0.1573896, -0.016781857, 0.11430...
1    [0.047489658, -0.25982162, 0.009633851, 0.1504...
2    [-0.0013053766, -0.2179021, -0.10905595, 0.126...
3    [-0.02052881, -0.31195918, -0.04454572, 0.1752...
4    [0.16468629, -0.41880217, 0.13673343, 0.298159...
Name: w2v, dtype: object

In [253]:
x_train, x_test, y_train, y_test = train_test_split(X, df_y, test_size=0.2, random_state=4)
mnb = MultinomialNB(alpha=0.1)
clf=mnb.fit(x_train,y_train)

ValueError: setting an array element with a sequence.

## Topic modeling

In [310]:
big_frame['tokenized_column'] = big_frame[0].fillna("").map(word_tokenize)
stop = stopwords.words('english')
x=big_frame['tokenized_column'].apply(lambda x: [item for item in x if item not in stop])


In [309]:
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [311]:
from gensim.models import ldamodel

num_topics = 8
id2word = gensim.corpora.Dictionary(x)
corpus = [id2word.doc2bow(text) for text in x]
lda = ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics)

In [312]:
def get_lda_topics(model, num_topics):
    word_dict = {};
    for i in range(num_topics):
        words = model.show_topic(i, topn = 40);
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words];
    return pd.DataFrame(word_dict)

In [313]:
get_lda_topics(lda, num_topics)


,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08
0,.,.,),?,.,.,.,(
1,The,„,.,.,Dick,I,Gutenberg,.
2,work,The,$,#,“,n't,Project,}
3,'ll,works,[,:,'s,said,tm,>
4,went,keep,],said,',',Foundation,{
5,little,side,’,I,The,'s,back,new
6,away,stood,«,ye,Indians,:,Literary,To
7,He,one,”,thought,Joe,?,Archive,So
8,found,sat,‘,boy,dog,know,:,looked
9,came,moment,<,come,state,could,http,The
